In [415]:
# !pip install pandas
import pandas as pd

In [416]:
file_path = '../data/concat-csv.csv'
lacentral_df = pd.read_csv(file_path, sep=';')
lacentral_df.shape

(8016, 26)

In [417]:
lacentral_df.drop_duplicates(inplace=True)
lacentral_df['owners'] = lacentral_df['owners'].fillna(0)
lacentral_df['internalColor'] = lacentral_df['internalColor'].fillna('Autres')
lacentral_df['price'] = lacentral_df['price'].str.strip().str.extract(r'([0-9 ]+)')[0].str.replace(' ', '').astype('int')
lacentral_df['price'] = lacentral_df[lacentral_df['price'] < 200000]['price']
lacentral_df['horsepower'] = lacentral_df[lacentral_df['horsepower'] < 750]['horsepower']
lacentral_df['kilometers'] = lacentral_df['kilometers'].str.strip().str.extract(r'([0-9 ]+)')[0].str.replace(' ', '').astype('int')
lacentral_df['kilometers'] = lacentral_df[lacentral_df['kilometers'] < 350000]['kilometers']
lacentral_df.dropna(inplace=True)
if lacentral_df.isnull().any().any():
    print("There are null values in the DataFrame")
    print(lacentral_df.isna().sum())
else:
    print("There are no null values in the DataFrame")

There are no null values in the DataFrame


In [418]:

lacentral_df['brand'] = lacentral_df['model'].str.strip().str.split().str[0]
lacentral_df['energy'] = lacentral_df['energy'].replace(['Bicarburation essence bioéthanol', pd.NaT, 'Bicarburation essence / gpl'],'Autres')
# lacentral_df['energy'] = lacentral_df['energy'].replace({'Essence' : 0, 'Diesel' : 1, 'Hybrides' : 2, 'Électrique' : 3, 'Autres' : 4})
# lacentral_df['gear_box'] = lacentral_df['gear_box'].replace({'Automatique' : 0, 'Manuelle' : 1})
lacentral_df['zip_code'] = lacentral_df['zip_code'].str.strip().str.extract(r'([0-9 ]+)')[0].str.replace(' ', '').astype('int')
# lacentral_df['origin'] = lacentral_df['origin'].replace({'France' : 0, 'Importé' : 1})
# lacentral_df['technicalControl'] = lacentral_df['technicalControl'].replace({'Non requis' : 0, 'Requis' : 1})
# lacentral_df['firstHand'] = lacentral_df['firstHand'].replace({'Non' : 0, 'Oui' : 1})
lacentral_df['length'] = lacentral_df['length'].str.strip().str.extract(r'([0-9, ]+)')[0].str.replace(',', '.').str.replace(' ', '').astype('float')
# lacentral_df['trunkVolumeRange'] = lacentral_df['trunkVolumeRange'].replace({'Petit coffre' : 0, 'Moyen Coffre' : 1, 'Grand coffre' : 2})
# lacentral_df['pollutionNorm'] = lacentral_df['pollutionNorm'].replace({'EURO2' : 0, 'EURO3' : 1, 'EURO4' : 2, 'EURO5' : 3, 'EURO6' : 4})
lacentral_df['critAir'] = lacentral_df['critAir'].str.strip().str.replace('Euro 1 et avant', '5').str.replace(' ', '').str.extract(r'([0-9, ]+)')[0].astype('int')
lacentral_df['ratedHorsePower'] = lacentral_df['ratedHorsePower'].str.strip().str.extract(r'([0-9 ]+)')[0].str.replace(' ', '').astype('int')
# lacentral_df['co2'] = lacentral_df['co2'].replace({'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3, 'E' : 4, 'F' : 5, 'G' : 6})
print(lacentral_df.shape)

if lacentral_df.isnull().any().any():
    print("There are null values in the DataFrame")
    print(lacentral_df.isna().sum())
else:
    print("There are no null values in the DataFrame")

(6874, 27)
There are no null values in the DataFrame


In [419]:
list = lacentral_df.brand.value_counts().reset_index().brand.to_list()
print(list)
brand_origin = {
    'ABARTH': 'Italie',
    'ALFA': 'Italie',
    'ALPINE': 'France',
    'ASTON': 'Royaume-Uni',
    'AUDI': 'Allemagne',
    'AUSTIN': 'Royaume-Uni',
    'BENTLEY': 'Royaume-Uni',
    'BMW': 'Allemagne',
    'CHATENET': 'France',
    'CHEVROLET': 'États-Unis',
    'CHRYSLER': 'États-Unis',
    'CITROEN': 'France',
    'CUPRA': 'Espagne',
    'DACIA': 'Roumanie',
    'DAIHATSU': 'Japon',
    'DODGE': 'États-Unis',
    'DS': 'France',
    'FERRARI': 'Italie',
    'FIAT': 'Italie',
    'FORD': 'États-Unis',
    'HONDA': 'Japon',
    'HUMMER': 'États-Unis',
    'HYUNDAI': 'Corée du Sud',
    'INFINITI': 'Japon',
    'ISUZU': 'Japon',
    'IVECO': 'Italie',
    'JAGUAR': 'Royaume-Uni',
    'JEEP': 'États-Unis',
    'KIA': 'Corée du Sud',
    'LAMBORGHINI': 'Italie',
    'LANCIA': 'Italie',
    'LAND': 'Royaume-Uni',
    'LEXUS': 'Japon',
    'LIGIER': 'France',
    'LOTUS': 'Royaume-Uni',
    'LYNK': 'Chine',
    'MASERATI': 'Italie',
    'MAZDA': 'Japon',
    'MERCEDES': 'Allemagne',
    'MERCEDES-AMG': 'Allemagne',
    'MG': 'Royaume-Uni',
    'MG4': 'Chine',
    'MINI': 'Royaume-Uni',
    'MITSUBISHI': 'Japon',
    'NISSAN': 'Japon',
    'OPEL': 'Allemagne',
    'PEUGEOT': 'France',
    'PORSCHE': 'Allemagne',
    'RADICAL': 'Royaume-Uni',
    'RENAULT': 'France',
    'SAAB': 'Suède',
    'SEAT': 'Espagne',
    'SKODA': 'République tchèque',
    'SMART': 'Allemagne',
    'SSANGYONG': 'Corée du Sud',
    'SUBARU': 'Japon',
    'SUZUKI': 'Japon',
    'TESLA': 'États-Unis',
    'TOYOTA': 'Japon',
    'TRIUMPH': 'Royaume-Uni',
    'VOLKSWAGEN': 'Allemagne',
    'VOLVO': 'Suède'
}

# missing_brands = set(list) - set(brand_origin.keys())
# print(missing_brands)

lacentral_df['brand_origin'] = lacentral_df['brand'].map(brand_origin)

['PEUGEOT', 'RENAULT', 'VOLKSWAGEN', 'CITROEN', 'AUDI', 'MERCEDES', 'BMW', 'DACIA', 'TOYOTA', 'MINI', 'NISSAN', 'OPEL', 'FORD', 'FIAT', 'DS', 'KIA', 'SKODA', 'SEAT', 'HYUNDAI', 'PORSCHE', 'VOLVO', 'LAND', 'JEEP', 'MAZDA', 'ALFA', 'JAGUAR', 'SUZUKI', 'HONDA', 'SMART', 'LEXUS', 'MITSUBISHI', 'CHEVROLET', 'CUPRA', 'ABARTH', 'ALPINE', 'TESLA', 'SUBARU', 'LYNK', 'FERRARI', 'SAAB', 'MERCEDES-AMG', 'MG', 'LAMBORGHINI', 'MG4', 'LOTUS', 'ASTON', 'SSANGYONG']


In [420]:
lacentral_df

,price,model,energy,gear_box,kilometers,year,zip_code,cylinder,horsepower,origin,...,length,trunkVolumeRange,ratedHorsePower,pollutionNorm,critAir,co2,picture,page,brand,brand_origin
0,12480.0,CITROEN C3 III,Essence,Manuelle,128750.0,2020,1,1.2,68.0,France,...,4.00,Grand coffre,4,EURO6,1,A,https://image-annonce.lacentrale.fr/480x369/E1...,1,CITROEN,France
1,57660.0,BMW X5 G05,Diesel,Automatique,140501.0,2020,1,3.0,265.0,France,...,4.92,Moyen Coffre,16,EURO6,2,D,https://image-annonce.lacentrale.fr/480x369/E1...,1,BMW,Allemagne
2,68900.0,BMW X6 F16,Diesel,Automatique,66045.0,2019,45,3.0,313.0,France,...,4.91,Moyen Coffre,20,EURO6,2,E,https://image-annonce.lacentrale.fr/480x369/E1...,1,BMW,Allemagne
3,81730.0,AUDI Q8,Diesel,Automatique,29430.0,2020,76,3.0,231.0,France,...,4.99,Moyen Coffre,14,EURO6,2,E,https://image-annonce.lacentrale.fr/480x369/E1...,1,AUDI,Allemagne
4,35060.0,TOYOTA YARIS CROSS,Hybrides,Automatique,18.0,2024,27,1.5,116.0,France,...,4.18,Petit coffre,5,EURO6,1,B,https://image-annonce.lacentrale.fr/480x369/E1...,1,TOYOTA,Japon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,12640.0,AUDI A3 II CABRIOLET phase 3,Diesel,Manuelle,172840.0,2011,92,2.0,140.0,France,...,4.24,Moyen Coffre,7,EURO5,2,B,https://image-annonce.lacentrale.fr/480x369/W1...,501,AUDI,Allemagne
8012,12080.0,TOYOTA VERSO-S,Essence,Automatique,140191.0,2016,6,1.3,99.0,France,...,3.99,Moyen Coffre,5,EURO5,1,B,https://image-annonce.lacentrale.fr/480x369/E1...,501,TOYOTA,Japon
8013,28730.0,AUDI Q3 phase 2,Essence,Automatique,79350.0,2018,72,1.4,150.0,France,...,4.39,Moyen Coffre,8,EURO6,1,C,https://image-annonce.lacentrale.fr/480x369/W1...,501,AUDI,Allemagne
8014,13760.0,RENAULT MEGANE IV,Diesel,Manuelle,85843.0,2020,44,1.5,95.0,France,...,4.36,Moyen Coffre,4,EURO6,2,A,https://image-annonce.lacentrale.fr/480x369/E1...,501,RENAULT,France


In [421]:
save_path = '../data/lacentrale-dev-clean.csv'
lacentral_df.to_csv(save_path, index=False)

In [422]:
lacentral_df.dtypes

price                   float64
model                    object
energy                   object
gear_box                 object
kilometers              float64
year                      int64
zip_code                  int64
cylinder                float64
horsepower              float64
origin                   object
firstCirculationDate     object
technicalControl         object
firstHand                object
owners                  float64
externalColor            object
internalColor            object
doors                   float64
seats                   float64
length                  float64
trunkVolumeRange         object
ratedHorsePower           int64
pollutionNorm            object
critAir                   int64
co2                      object
picture                  object
page                      int64
brand                    object
brand_origin             object
dtype: object

In [423]:
# !pip install keras scipy scikit-learn
from sklearn.preprocessing import OrdinalEncoder

encoded_df = lacentral_df.drop(['picture','firstCirculationDate', 'page'], axis=1)
columns = ['model', 'externalColor', 'internalColor', 'brand', 'brand_origin', 'energy', 'gear_box', 'origin', 'technicalControl', 'firstHand', 'trunkVolumeRange', 'pollutionNorm', 'co2']
encoder = OrdinalEncoder()
relational_df = lacentral_df

for column in columns:
    encoded_df[[column]] = encoder.fit_transform(encoded_df[[column]])
    relational_df[[column+'_ecoded']] = encoded_df[[column]]

# relational_df.to_csv('../data/dataset_relational.csv', index=False)
encoded_df.to_csv('../data/lacentrale-dev-clean-encoded.csv', index=False)
encoded_df


,price,model,energy,gear_box,kilometers,year,zip_code,cylinder,horsepower,origin,...,doors,seats,length,trunkVolumeRange,ratedHorsePower,pollutionNorm,critAir,co2,brand,brand_origin
0,12480.0,185.0,2.0,1.0,128750.0,2020,1,1.2,68.0,0.0,...,5.0,2.0,4.00,0.0,4,4.0,1,0.0,7.0,4.0
1,57660.0,159.0,1.0,0.0,140501.0,2020,1,3.0,265.0,0.0,...,5.0,7.0,4.92,1.0,16,4.0,2,3.0,5.0,0.0
2,68900.0,162.0,1.0,0.0,66045.0,2019,45,3.0,313.0,0.0,...,5.0,5.0,4.91,1.0,20,4.0,2,4.0,5.0,0.0
3,81730.0,71.0,1.0,0.0,29430.0,2020,76,3.0,231.0,0.0,...,5.0,5.0,4.99,1.0,14,4.0,2,4.0,4.0,0.0
4,35060.0,834.0,3.0,0.0,18.0,2024,27,1.5,116.0,0.0,...,5.0,5.0,4.18,2.0,5,4.0,1,1.0,44.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,12640.0,16.0,1.0,1.0,172840.0,2011,92,2.0,140.0,0.0,...,2.0,4.0,4.24,1.0,7,3.0,2,1.0,4.0,0.0
8012,12080.0,833.0,2.0,0.0,140191.0,2016,6,1.3,99.0,0.0,...,5.0,5.0,3.99,1.0,5,3.0,1,1.0,44.0,6.0
8013,28730.0,62.0,2.0,0.0,79350.0,2018,72,1.4,150.0,0.0,...,5.0,5.0,4.39,1.0,8,4.0,1,2.0,4.0,0.0
8014,13760.0,716.0,1.0,1.0,85843.0,2020,44,1.5,95.0,0.0,...,5.0,5.0,4.36,1.0,4,4.0,2,0.0,35.0,4.0


In [424]:
relational_df.to_csv('../data/dataset_relational.csv', index=False)
relational_df

,price,model,energy,gear_box,kilometers,year,zip_code,cylinder,horsepower,origin,...,brand_ecoded,brand_origin_ecoded,energy_ecoded,gear_box_ecoded,origin_ecoded,technicalControl_ecoded,firstHand_ecoded,trunkVolumeRange_ecoded,pollutionNorm_ecoded,co2_ecoded
0,12480.0,CITROEN C3 III,Essence,Manuelle,128750.0,2020,1,1.2,68.0,France,...,7.0,4.0,2.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0
1,57660.0,BMW X5 G05,Diesel,Automatique,140501.0,2020,1,3.0,265.0,France,...,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,3.0
2,68900.0,BMW X6 F16,Diesel,Automatique,66045.0,2019,45,3.0,313.0,France,...,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0
3,81730.0,AUDI Q8,Diesel,Automatique,29430.0,2020,76,3.0,231.0,France,...,4.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0
4,35060.0,TOYOTA YARIS CROSS,Hybrides,Automatique,18.0,2024,27,1.5,116.0,France,...,44.0,6.0,3.0,0.0,0.0,0.0,0.0,2.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,12640.0,AUDI A3 II CABRIOLET phase 3,Diesel,Manuelle,172840.0,2011,92,2.0,140.0,France,...,4.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,3.0,1.0
8012,12080.0,TOYOTA VERSO-S,Essence,Automatique,140191.0,2016,6,1.3,99.0,France,...,44.0,6.0,2.0,0.0,0.0,1.0,0.0,1.0,3.0,1.0
8013,28730.0,AUDI Q3 phase 2,Essence,Automatique,79350.0,2018,72,1.4,150.0,France,...,4.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,4.0,2.0
8014,13760.0,RENAULT MEGANE IV,Diesel,Manuelle,85843.0,2020,44,1.5,95.0,France,...,35.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0


In [425]:
def custom_encoder(col):
    return {
        'decoded' : data[str(col)].unique().tolist(),
        'encoded' : data[str(col)+'_ecoded'].unique().tolist()
    }


custom_encoder('co2')

{'decoded': [0.0, 3.0, 4.0, 1.0, 5.0, 2.0, 6.0],
 'encoded': [0.0, 3.0, 4.0, 1.0, 5.0, 2.0, 6.0]}